# USDA Data Exploration
USDA’s Foreign Agricultural Service data API portal provides users the ability to obtain programmatic access to publicly available agricultural commodity data from the Export Sales Report (ESR), Global Agricultural Trade System (GATS), and Production, Supply & Distribution (PSD) databases. This notebook explores ESR and PSD data. More information about these data sources [can be found here.](https://apps.fas.usda.gov/opendataweb/about)

In [2]:
# general imports
import cmath
import numpy as np
import pandas as pd
import streamlit as st
import requests
import geopandas as gpd
from datetime import date

## USDA Export Sales Report (ESR) Data
USDA's Export Sales Reporting Program monitors U.S. agricultural export sales on a daily and weekly basis. Export sales reporting provides a constant stream of up-to-date market information for 40 U.S. agricultural commodities sold abroad.

A single statistic reveals the significance of the program: in a typical year, the program monitors more than 40 percent of total U.S. agricultural exports. The program also serves as an early alert on the possible impact foreign sales have on U.S. supplies and prices.

The weekly U.S. Export Sales report is the most currently available source of U.S. export sales data. The data is used to analyze the overall level of export demand, determine where markets exist, and assess the relative position of U.S. commodities in foreign markets.

In [8]:
from requests.auth import HTTPBasicAuth
# function that gets data from USDA endpoint and converts response to pandas dataframe
def get_usda_data(endpoint_url):
    # USDA API Key
    key = {"API_KEY": "aefd68b9-cfdc-4c9e-a800-b457ff5adade"}
    request = requests.get(url=endpoint_url, headers=key)
    # convert response to pandas dataframe
    return (pd.read_json(request.text))


In [5]:
# Basic endpoints. See below for endpoints that return data commodity and country specific data
regions_url = "https://apps.fas.usda.gov/OpenData/api/esr/regions"
country_url = "https://apps.fas.usda.gov/OpenData/api/esr/countries"
data_release_url = "https://apps.fas.usda.gov/OpenData/api/esr/datareleasedates"
commodity_url = "https://apps.fas.usda.gov/OpenData/api/esr/commodities"
units_url = "https://apps.fas.usda.gov/OpenData/api/esr/unitsOfMeasure"


In [6]:
# Regions with Region ID's
get_usda_data(regions_url)

,regionId,regionName
0,1,EUROPEAN UNION - 27
1,2,OTHER EUROPE
2,3,EASTERN EUROPE
3,4,FORMER SOVIET UNION-12
4,5,JAPAN
5,6,TAIWAN
6,7,CHINA
7,8,INDIA
8,9,OTHER ASIA AND OCEANIA
9,10,AFRICA


In [60]:
get_usda_data(country_url)

,countryCode,countryName,countryDescription,regionId,gencCode
0,1,EUROPEAN,EUROPEAN UNION - 27,1,None
1,2,UNKNOWN,UNKNOWN,99,AX1
2,1010,GREENLD,GREENLAND,11,GRL
3,1220,CANADA,CANADA,11,CAN
4,1610,MIGUEL,ST. PIERRE AND MIQUELON,11,None
...,...,...,...,...,...
205,9350,GUAM,GUAM,9,GUM
206,9410,CAN EN I,CANTON AND ENDERBURY ISLANDS,9,None
207,9510,AM SAMOA,AMERICAN SAMOA,9,ASM
208,9610,NMARIANA,NORTHERN MARIANA ISLANDS,9,MNP


In [7]:
get_usda_data(commodity_url)

,commodityCode,commodityName,unitId
0,101,Wheat - HRW,1
1,102,Wheat - SRW,1
2,103,Wheat - HRS,1
3,104,Wheat - White,1
4,105,Wheat - Durum,1
5,106,Wheat - Mixed,1
6,107,All Wheat,1
7,201,Wheat Products,1
8,301,Barley,1
9,401,Corn,1


In [75]:
get_usda_data(data_release_url)

,commodityCode,marketYearStart,marketYearEnd,marketYear,releaseTimeStamp
0,101,2022-06-01T00:00:00,2023-05-31T00:00:00,2023,2022-07-28T00:00:00
1,102,2022-06-01T00:00:00,2023-05-31T00:00:00,2023,2022-07-28T00:00:00
2,103,2022-06-01T00:00:00,2023-05-31T00:00:00,2023,2022-07-28T00:00:00
3,104,2022-06-01T00:00:00,2023-05-31T00:00:00,2023,2022-07-28T00:00:00
4,105,2022-06-01T00:00:00,2023-05-31T00:00:00,2023,2022-07-28T00:00:00
...,...,...,...,...,...
999,1602,1998-01-01T00:00:00,1998-12-31T00:00:00,1998,1999-01-07T00:00:00
1000,1603,1998-01-01T00:00:00,1998-12-31T00:00:00,1998,1999-01-07T00:00:00
1001,1606,1998-01-01T00:00:00,1998-12-31T00:00:00,1998,1999-01-07T00:00:00
1002,1607,1998-01-01T00:00:00,1998-12-31T00:00:00,1998,1999-01-07T00:00:00


In [63]:
# Units of commodities referenced in chart above.
get_usda_data(units_url)

,unitId,unitNames
0,1,Metric Tons
1,2,Running Bales
2,3,Pieces
3,4,NUMBER
4,5,Pounds


Given Commodity Code (Ex: 107 for "All Wheat") and MarketYear (Ex: 2021) this API End point will return a list of US Export records of White Wheat to all applicable countries from USA for the given Market Year. See DataReleaseDates end point to get a list of all Commodities and the corresponding Market Year data.

In [70]:
# return first 5 rows for "All Wheat" in 2021
wheat_url = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/107/allCountries/marketYear/2021"
get_usda_data(wheat_url).head(5)

,commodityCode,countryCode,weeklyExports,accumulatedExports,outstandingSales,grossNewSales,currentMYNetSales,currentMYTotalCommitment,nextMYOutstandingSales,nextMYNetSales,unitId,weekEndingDate
0,107,1220,0,0,9266,9266,0,9266,0,0,1,2020-06-04T00:00:00
1,107,2010,13506,13506,352846,224335,25376,366352,0,0,1,2020-06-04T00:00:00
2,107,2050,0,0,95999,95999,95999,95999,0,0,1,2020-06-04T00:00:00
3,107,2080,0,0,4000,0,0,4000,0,0,1,2020-06-04T00:00:00
4,107,2110,0,0,10000,0,0,10000,0,0,1,2020-06-04T00:00:00


Given Commodity Code (Ex: 107 for "All Wheat"), Country Code (Ex:1220 for Canada) and MarketYear (Ex: 2021) this API End point will return a list of US Export records of White Wheat to Canada from USA for the give Market Year. Please see DataReleaseDates end point to get a list of all Commodities and the corresponding Market Year data.

In [73]:
# return first 5 rows for "All Wheat" in 2021
canada_wheat_url = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/107/countryCode/2021/marketYear/1220"
get_usda_data(wheat_url).head(5)

,commodityCode,countryCode,weeklyExports,accumulatedExports,outstandingSales,grossNewSales,currentMYNetSales,currentMYTotalCommitment,nextMYOutstandingSales,nextMYNetSales,unitId,weekEndingDate
0,107,1220,0,0,9266,9266,0,9266,0,0,1,2020-06-04T00:00:00
1,107,2010,13506,13506,352846,224335,25376,366352,0,0,1,2020-06-04T00:00:00
2,107,2050,0,0,95999,95999,95999,95999,0,0,1,2020-06-04T00:00:00
3,107,2080,0,0,4000,0,0,4000,0,0,1,2020-06-04T00:00:00
4,107,2110,0,0,10000,0,0,10000,0,0,1,2020-06-04T00:00:00


## Production, Supply & Distribution (PSD) Data
PSD Online is the public repository for USDA’s Official Production, Supply and Distribution forecast data, reports and circulars for key agricultural commodities.

FAS's PSD Online data are reviewed and updated monthly by an interagency committee chaired by USDA's World Agricultural Outlook Board (WAOB),and consisting of: the Foreign Agricultural Service (FAS), the Economic Research Service (ERS),the Farm Service Agency (FSA), and the Agricultural Marketing Service (AMS).

The international portion of the data is updated with input from agricultural attachés stationed at U.S. embassies around the world, FAS commodity analysts, and country and commodity analysts with ERS. The U.S. domestic component is updated with input from analysts in FAS, ERS, the National Agricultural Statistical Service, and FSA.

*Note, country and commodity codes are differen that ESR data*

In [105]:
psd_com_attributes_url = "https://apps.fas.usda.gov/OpenData/api/psd/commodityAttributes"
psd_units_url = "https://apps.fas.usda.gov/OpenData/api/psd/unitsOfMeasure"
psd_com_url = "https://apps.fas.usda.gov/OpenData/api/psd/commodities"
psd_countries_url = "https://apps.fas.usda.gov/OpenData/api/psd/countries"
psd_regions_url = "https://apps.fas.usda.gov/OpenData/api/psd/regions"


In [11]:
get_usda_data(psd_com_attributes_url)

NameError: name 'psd_com_attributes_url' is not defined

In [102]:
get_usda_data(psd_units_url)

,unitId,unitDescription
0,1,(1000 BUSHES)
1,2,(1000 60 KG BAGS)
2,3,(1000 COLONIES)
3,4,(1000 HA)
4,5,(1000 HEAD)
5,6,(1000 HL)
6,7,(1000 MT CWE)
7,8,(1000 MT)
8,9,(1000 PCS)
9,10,(1000 TREES)


In [104]:
get_usda_data(psd_com_url)

,commodityCode,commodityName
0,577400,"Almonds, Shelled Basis"
1,11000,"Animal Numbers, Cattle"
2,13000,"Animal Numbers, Swine"
3,574000,"Apples, Fresh"
4,430000,Barley
5,579305,"Cherries (Sweet&Sour), Fresh"
6,711100,"Coffee, Green"
7,440000,Corn
8,2631000,Cotton
9,230000,"Dairy, Butter"


In [ ]:
get_usda_data(psd_regions_url)

In [ ]:
get_usda_data(psd_countries_url)

Returns a set of records with Forecast number for a given Commodity Code (Ex, 0440000 for Corn) and a given Market Year (Ex, 2017) for all applicable countries. PSD Database has data for most commodities since year 1960. Please see Commodities end point to get a list of all PSD Commodities with corresponding Commodity Code.

In [129]:
psd_comyear_url = "https://apps.fas.usda.gov/OpenData/api/psd/commodity/{commodityCode}/country/all/year/{marketYear}"
get_usda_data(psd_comyear_url.format(commodityCode = "0440000", marketYear = "2021"))

,commodityCode,countryCode,marketYear,calendarYear,month,attributeId,unitId,value
0,440000,AF,2021,2022,5,4,4,130.0000
1,440000,AF,2021,2022,5,20,8,0.0000
2,440000,AF,2021,2022,5,28,8,220.0000
3,440000,AF,2021,2022,5,57,8,5.0000
4,440000,AF,2021,2022,5,81,8,5.0000
...,...,...,...,...,...,...,...,...
1870,440000,ZA,2021,2022,7,192,8,2100.0000
1871,440000,ZA,2021,2022,7,125,8,2800.0000
1872,440000,ZA,2021,2022,7,176,8,971.0000
1873,440000,ZA,2021,2022,7,178,8,4471.0000


Returns a set of records with Forecast number for a given Commodity Code (Ex, 0440000 for Corn) and a given Market Year (Ex, 2017) for a given Country (Ex, BR for Brazil). PSD Database has data for most commodities since year 1960. Please see Commodities end point to get a list of all PSD Commodities with corresponding Commodity Code, and Countries end point for a list of all countries with corresponding Country Code

In [128]:
psd_comcountryyear_url = "https://apps.fas.usda.gov/OpenData/api/psd/commodity/{commodityCode}/country/{countryCode}/year/{marketYear}"
get_usda_data(psd_comcountryyear_url.format(commodityCode = "0440000", countryCode = "BR", marketYear = "2021"))

,commodityCode,countryCode,marketYear,calendarYear,month,attributeId,unitId,value
0,440000,BR,2021,2022,6,4,4,21800.0000
1,440000,BR,2021,2022,6,20,8,4153.0000
2,440000,BR,2021,2022,6,28,8,116000.0000
3,440000,BR,2021,2022,6,57,8,2000.0000
4,440000,BR,2021,2022,6,81,8,2500.0000
5,440000,BR,2021,2022,6,84,8,0.0000
6,440000,BR,2021,2022,6,86,8,122153.0000
7,440000,BR,2021,2022,6,88,8,44500.0000
8,440000,BR,2021,2022,6,113,8,34000.0000
9,440000,BR,2021,2022,6,130,8,62000.0000


Returns a set of records with Forecast number for a given Commodity Code (Ex, 0440000 for Corn) and a given Market Year (Ex, 2017) for the world.Data from all applicable countries are aggregated together for reporting it at the World Level. PSD Database has data for most commodities since year 1960. Please see Commodities end point to get a list of all PSD Commodities with corresponding Commodity Code.

In [127]:
psd_comworldyear_url = "https://apps.fas.usda.gov/OpenData/api/psd/commodity/{commodityCode}/world/year/{marketYear}"
get_usda_data(psd_comworldyear_url.format(commodityCode = "0440000", marketYear = "2021"))

,commodityCode,countryCode,marketYear,calendarYear,month,attributeId,unitId,value
0,440000,0,2021,2022,7,4,4,2.067390e+05
1,440000,0,2021,2022,7,20,8,2.932910e+05
2,440000,0,2021,2022,7,28,8,1.217866e+06
3,440000,0,2021,2022,7,57,8,1.791450e+05
4,440000,0,2021,2022,7,81,8,1.813980e+05
5,440000,0,2021,2022,7,84,8,0.000000e+00
6,440000,0,2021,2022,7,86,8,1.690302e+06
7,440000,0,2021,2022,7,88,8,1.992330e+05
8,440000,0,2021,2022,7,113,8,1.899280e+05
9,440000,0,2021,2022,7,130,8,7.462700e+05


In [10]:
gro_url = "https://community.gro-intelligence.com/food-security-tracker-africa/data/Sudan_Wheat_supply_demand.json"
get_usda_data(gro_url)
# See chart of data on website here: https://community.gro-intelligence.com/food-security-tracker-africa/details?country=Sudan&crop=Wheat

,Production Quantity,Domestic Consumption,USDA Stocks to Use Ratio,Gro Implied Stocks to Use Ratio
2013/14,265,2825,17.391304,NaN
2014/15,473,3125,15.104000,NaN
2015/16,779,2950,10.949153,NaN
2016/17,516,3000,9.900000,NaN
2017/18,463,3050,9.508197,NaN
2018/19,702,3000,7.120076,NaN
2019/20,726,3100,16.903226,NaN
2020/21,751,2900,14.939759,NaN
2021/22,600,2850,10.508475,NaN
2022/23,700,2600,10.518234,11.529111


https://community.gro-intelligence.com/food-security-tracker-africa/details?country=Sudan&crop=Wheat

/Users/cameronkruse/Desktop/Screen Shot 2022-07-31 at 6.57.33 PM.png
